## Stacking Ensemble

<정의> 여러 개별 알고리즘 모델을 결합하여 도출한 예측 결과 데이터 세트를 각 각 스태킹 형태로 결합해, 최종 메타 모델의 학습용, 테스트용 feature dataset 을 만든 뒤 별도의 ML 알고리즘으로 최종 학습을 수행하고 테스트 데이터를 기반으로 다시 최종 예측을 수행하는 방식. 



* 캐글과 같은 대회에서 높은 순위를 차지하기 위해 조금이라도 성능 수치를 높여야 할 경우 자주 사용됨!<br><br>
* <자세한 설명> [이미지 자료](https://media.vlpt.us/images/dbj2000/post/8cf8374b-4ea6-437d-a0b3-92276af08abc/image.png)<br>
     - M x N dataset 에서 ML 알고리즘 모델별로 각 각 학습 시킨 뒤 예측을 수행하면 각 각 M 개의 row를 가진 1개의 label 값을 도출<br>
     - 도출된 예측 label 값을 합해서(stacking) 새로운 dataset 을 만듦<br>
     - 해당 dataset 을 최종 모델에 학습, 예측 함.

<예제> 내장 dataset 위스콘신 유방암 데이터 사용 

In [27]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_feature = cancer_data.data
y_label = cancer_data.target

X_train,X_test,y_train,y_test = train_test_split(X_feature,y_label,test_size=0.2, random_state = 22)

# Stacking 에 사용될 알고리즘 모델 생성 
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100,random_state=22)
ada_clf = AdaBoostClassifier(n_estimators=100)
df_clf = DecisionTreeClassifier()

# 최종 알고리즘 모델 
lr_clf = LogisticRegression(C=10)

# 학습 
knn_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
ada_clf.fit(X_train,y_train)
df_clf.fit(X_train,y_train)

# 예측 
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)
df_pred = df_clf.predict(X_test)

# 성능 평가 지표  ( Accuracy )
print('KNN 정확도:{0:.4f}'.format(accuracy_score(y_test,knn_pred)))
print('RandomForest 정확도:{0:.4f}'.format(accuracy_score(y_test,rf_pred)))
print('AdaBoost 정확도:{0:.4f}'.format(accuracy_score(y_test,ada_pred)))
print('DecisionTree 정확도:{0:.4f}'.format(accuracy_score(y_test,df_pred)))

KNN 정확도:0.9298
RandomForest 정확도:0.9649
AdaBoost 정확도:0.9737
DecisionTree 정확도:0.9123


이렇게 개별 알고리즘으로 부터 예측된 예측값을 칼럼 레벨로 옆으로 붙여서 feature 값으로 만들어, 최종 메타 모델인 LogisticRegression 에서 학습 데이터로 다시 사용한다. <br><br>
반환된 값은 label 예측 값을 가진 1차원 ndarray 이므로 행 행태로 붙인 뒤 transpose 해주자.

In [25]:
pred = np.array([knn_pred,rf_pred,df_pred,ada_pred])
print(pred.shape)

pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [28]:
import warnings
warnings.filterwarnings('ignore')

# 최종 알고리즘 모델 LogisticRegression 에 적용 
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)
print("LogisticRegression 정확도: {0:.4f}".format(accuracy_score(y_test,lr_pred)))

LogisticRegression 정확도: 0.9561


개별 알고리즘 모델 중 KNN 과 DecisionTree 보다는 정확도가 향상되었지만 RandomForest 와 AdaBoost 보다는 떨어졌다.<br>
즉, 스태킹 기법이라 해서 무조건 개별 모델보다 성능이 좋아지는 것만은 아니다.
## CV 세트 기반의 Stacking  ( 어려움 )

과적합을 개선하기 위한 CV 세트 기반의 Stacking 을 알아보자. ( p.282 ~ 288 )<br><br>
<정의> 여러 개별 알고리즘 모델들이 각 각의 교차 검증으로 도출한 학습용 예측 label 값과 테스트용 예측 label 값들을 분할된 폴드 수에 맞게 반복하며 스태킹 형태로 합치고, 마지막으로 스태킹 형태로 합친 각 모델 별 학습용,테스트용 데이터들을 기반으로 최종 메타 모델이 학습과 예측을 수행한다. 

* <자세한 설명><br><br>
    * 1.[이미지 자료 1](https://media.vlpt.us/images/dbj2000/post/1d9611a2-2ca4-4516-8ffe-562e044d5b47/image.png)<br>
    먼저 학습용 데이터를 N개의 폴드로 나눈다. 그 때 N-1 개 폴드는 학습 데이터를 기반으로 개별 모델을 학습 시킨다. <br><u>검증 폴드 1개 데이터</u>로 예측을 진행하고 결과를 저장하며 이 저장된 예측 결과값은 __최종 메타 모델에서 학습 데이터__ 로 사용된다. <br>N-1 개의 폴드로 학습한 모델은 <u>원본 테스트 세트</u>로도 예측을 진행하고 해당 결과를 저장하며 이 저장된 예측 결과값은 __최종 메타 모델에서 테스트 데이터__ 로 사용된다.<br><br>
    * 2.[이미지 자료 2](https://media.vlpt.us/images/dbj2000/post/73814738-3d92-4412-9f5e-36e73ca04222/image.png) [ 이미지 자료 3](https://media.vlpt.us/images/dbj2000/post/5a65a534-ee72-48dc-9efb-f317d4ebf9c6/image.png)<br> 학습 데이터와 검증 데이터 세트를 변경해 가면서 반복한다. 검증 폴드로 한 예측의 반복된 결과값들은 합쳐주고, 원본 테스트 데이터로 한 예측의 반복된 결과값들은 평균 해주어서 최종 메타 모델에서 사용된다.<br><br>
    * 3.[이미지 자료 4](https://media.vlpt.us/images/dbj2000/post/3599e0ec-52df-40ac-bf29-228cd39890fb/image.png)<br> 마지막으로 1 ~ 2 과정을 거친 모든 모델의 두 결과값들을 모두 합쳐서 최종 메타 모델이 사용할 학습 데이터와 테스트 데이터를 생성한 뒤 최종 메타 모델에 학습, 예측을 진행하면 된다.
    
코드화 시켜보자.

In [30]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 모델에서 최종 메타 모델이 사용 할 학습용 , 테스트용 데이터를 생성하기 위한 함수 

# parameters : 개별 Classifier 모델 , 원본인 학습용 feature 데이터 , 원본인 학습용 label 데이터, 원본인 테스트 feature 데이터, K폴드 갯수 
def get_stacking_base_datasets(model,X_train_n,y_train_n,X_test_n, n_folds):
    
    # 지정된 n 값으로 폴드 객체 생성 
    kf = KFold(n_splits=n_folds,shuffle=False, random_state=11)
    
    # 추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0],1))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__,'model 시작')
    
    # Kfold 의 split 으로  학습 및 검증을 위한 데이터 세트의 인덱스를 자동으로 분할 
    for folder_counter, (train_index,valid_index) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측 할 폴드 데이터 세트 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 개별 모델의 학습 수행 
        model.fit(X_tr,y_tr)
        
        # 폴드 세트 내부에서 다시 만들어진 검증 데이터로 개별 모델 예측 후 데이터 저장 → 이게 최종 메타 모델의 학습 데이터로 사용
        train_fold_pred[valid_index,:]= model.predict(X_te).reshape(-1,1)
        
        # 입력된 원본 테스트 데이터를 폴드 세트 내 학습된 개별 모델에서 예측 후 데이터 저장 → 이게 최종 메타 모델의 테스트 데이터로 사용 
        test_pred[:,folder_counter]=model.predict(X_test_n)
    
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred,axis=1).reshape(-1,1)
    
    return train_fold_pred , test_pred_mean        

In [31]:
# 개별 모델에 하나씩 적용
knn_train , knn_test = get_stacking_base_datasets(knn_clf,X_train,y_train,X_test,7)
rf_train , rf_test = get_stacking_base_datasets(rf_clf,X_train,y_train,X_test,7)
dt_train , dt_test = get_stacking_base_datasets(df_clf,X_train,y_train,X_test,7)
ada_train , ada_test = get_stacking_base_datasets(ada_clf,X_train,y_train,X_test,7)

KNeighborsClassifier model 시작
	 폴드 세트:  0  시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
	 폴드 세트:  6  시작
RandomForestClassifier model 시작
	 폴드 세트:  0  시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
	 폴드 세트:  6  시작
DecisionTreeClassifier model 시작
	 폴드 세트:  0  시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
	 폴드 세트:  6  시작
AdaBoostClassifier model 시작
	 폴드 세트:  0  시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
	 폴드 세트:  6  시작


이제, 각 모델별로 도출해낸 최종 메타 모델을 위한 학습 데이터와 테스트 데이터를 합치기만 하면 된다.<br>
< concatenate() 은 여러 개의 넘파일 배열을 칼럼 또는 row level 로 합쳐주는 기능을 제공함 >

In [32]:
# 최종 메타 모델이 학습할 feature dataset
Stack_final_X_train = np.concatenate((knn_train,rf_train,dt_train,ada_train),axis=1)

# 최종 메타 모델이 테스트할 feature dataset
Stack_final_X_test = np.concatenate((knn_test,rf_test,dt_test,ada_test),axis=1)

마지막으로 최종 메타 모델인 LogisticRegression 으로 도출한 학습 데이터와 테스트 데이터를 이용해서 학습, 예측을 진행해보자.

In [36]:
lr_clf.fit(Stack_final_X_train,y_train)
logistic_pred = lr_clf.predict(Stack_final_X_test)
print("CV 세트 기반의 Stacking \n최종 메타 모델 LogisticRegression 예측 정확도 : {0:.4f}".format(accuracy_score(y_test,logistic_pred)))

CV 세트 기반의 Stacking 
최종 메타 모델 LogisticRegression 예측 정확도 : 0.9737


이로써 CV 세트로 분할하여 Stacking 을 진행해 보았다. 여기에서 개별 모델에 대한 하이퍼 파라미터 튜닝 GridSearchCV 를 이용하면<br> 성능이 개선 될 가능성이 있다 생각한다. 추후에 시도해보자. 